<h1>Phonetic Search Example<span class="tocSkip"></span></h1>
<div class="toc">
    <ul class="toc-item">
        <li>
        <span><a href="#Packages-and-functions" data-toc-modified-id="Packages-and-functions-1">
        <span class="toc-item-num">1&nbsp;&nbsp;</span>Packages and functions</a></span>
        </li>
        <li>
            <span><a href="#Prepare-and-match-admin-boundary-data" data-toc-modified-id="Prepare-and-match-admin-boundary-data-2">
            <span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare and match admin boundary data</a></span>
            <ul class="toc-item">
                <li>
                <span><a href="#Match-health-list-admin-names-to-shapefile-admin-names" data-toc-modified-id="Match-health-list-admin-names-to-shapefile-admin-names-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Match health list admin names to shapefile admin names</a></span>
                <ul class="toc-item">
                <li>
                <span><a href="#Match-orgunitlevel4-to-Geob-Adm-3" data-toc-modified-id="Match-orgunitlevel4-to-Geob-Adm-3-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Match orgunitlevel4 to Geob Adm 3</a></span>
                </li>
                <li>
                <span><a href="#Match-orgunitlevel3-to-Geob-Adm-2" data-toc-modified-id="Match-orgunitlevel3-to-Geob-Adm-2-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Match orgunitlevel3 to Geob Adm 2</a></span>
                </li>
                <li>
                <span><a href="#Match-orgunitlevel2-to-Geob-Adm-1" data-toc-modified-id="Match-orgunitlevel2-to-Geob-Adm-1-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Match orgunitlevel2 to Geob Adm 1</a></span>
                </li>
                </ul>
            </li>
        </ul>
    </li>
</div>

Version: April 11, 2023

This notebook works with health facilities from the HMIS Database and attempts to geolocate them, by joining them to administrative shapefiles, other sources of geo-located health facilities, and running queries with geocoding APIs.

**Data Sources**
- Health facilities (hierarchy list)
- Geoboundaries (Adm1 through 4)
- FEWS Admin-2 boundaries (updated post 2017)

# Packages and functions

In [1]:
import os
import geopandas as gpd
import jellyfish
from os.path import join
import pandas as pd
import regex as re
from string import ascii_uppercase
from thefuzz import fuzz


In [2]:
from ctypes.util import find_library
find_library('c')

'/usr/lib/libc.dylib'

# Prepare and match admin boundary data

In [3]:
def remove_words(data_frame, column_name, words_to_remove = ['community', 'clinic', 'centre', 'center', 'hospital', 'health', 'government']):
    """ Remove words from strings in a specified column

    Args:
        data_frame: Pandas data frame
        column_name: Column name to remove words from
        words_to_remove: List of words

    Returns:
        Removes words in place

    """
    for word in words_to_remove:
        data_frame[column_name] = data_frame[column_name].str.replace(word, "")
    data_frame[column_name] = data_frame[column_name].str.strip()

def report_geocoding(records):
    """ Report value counts and % from geocoding_method column.

    Args:
        records: Pandas data frame

    Returns:
        Table with count stats
    """
    table = records[['NAME', 'geocoding_method']].groupby('geocoding_method').count().rename(columns={'NAME':'count'})
    table.loc[:, "pct"] = table/(table['count'].sum())
    table = table.style.format({
        'pct': '{:,.1%}'.format,
    })
    return(table)

def match_name(name, list_names, min_score=0):
    """ Uses fuzzy matching between one string and a list of candidate names.

    Args:
        name: string
        list_names: list of candidate names to match
        min_score: minimum score allowed

    Returns:
        Returns name with the highest match, and match score
        If no match is higher than min score, returns "" and -1
    """
    # -1 score incase we don't get any matches
    max_score = -1
    # Returning empty name for no match as well
    max_name = ""
    # Iterating over all names in the other
    for name2 in list_names:

        # First try string matching - if we get an exact match, stop
        special_char_regex = r"[^\w\s\d]"
        multiple_spaces_regex = r"[\s]{2,}"
        regex_name = re.sub(multiple_spaces_regex, " ", re.sub(special_char_regex, '', name.lower()))
        regex_name2 = re.sub(multiple_spaces_regex, " ", re.sub(special_char_regex, '', name2.lower()))
        if  regex_name == regex_name2:
            max_name = name2
            max_score = 100
            print(f"Exact Match! [{regex_name}] == [{regex_name2}]")
            break
        # Finding fuzzy match score
        else:
            # get the phonetic score
            phon1 = jellyfish.metaphone(name) #.soundex(name)
            phon2 = jellyfish.metaphone(name2) #.soundex(name2)
            phon_score = fuzz.ratio(phon1, phon2)

            # get the fuzzy score
            fuzzy_score = fuzz.ratio(name, name2)

            print(f"{name} vs {name2}: phon: {phon_score}, fuzzy: {fuzzy_score}")
            winning_score = phon_score if (phon_score >= fuzzy_score and phon_score > 75) else fuzzy_score
            # Checking if we are above our threshold and have a better score
            if (winning_score > min_score) & (winning_score > max_score):
                max_name = name2
                max_score = winning_score

    return (max_name, max_score)



In [4]:
iso3 = "LBR"
country = "Liberia"
input_dir = "/Users/dianaholcomb/Documents/GWU/6501_Capstone/workspace/data"
output_dir = join(input_dir, "output", iso3)
# if not os.path.exists(out_folder):
#     os.mkdir(out_folder)

In [5]:
geob2 = gpd.read_file(f"https://github.com/wmgeolab/geoBoundaries/raw/b7dd6a5/releaseData/gbOpen/{iso3}/ADM2/geoBoundaries-{iso3}-ADM2.geojson")
geob1 = gpd.read_file(f"https://github.com/wmgeolab/geoBoundaries/raw/b7dd6a5/releaseData/gbOpen/{iso3}/ADM1/geoBoundaries-{iso3}-ADM1.geojson")

In [6]:
master_table = pd.read_csv(os.path.join(input_dir, "liberia2022-11-15.csv"), low_memory=False)

In [7]:
len(master_table)

72008

In [8]:
master_table.head(2)

,orgunitlevel1,orgunitlevel2,orgunitlevel3,orgunitlevel4,organisationunitid,organisationunitname,organisationunitcode,organisationunitdescription,periodid,periodname,periodcode,perioddescription,dataid,dataname,datacode,datadescription,Total,date_downloaded
0,Liberia,Sinoe,Tarjuwon District,Gbason Town Clinic,DAmhVvbzEl6,Gbason Town Clinic,LR5GZ3,NaN,202207,July 2022,202207,NaN,OEkmu8lM1lr,OPD new cases total,NaN,NaN,20.0,2022-11-15
1,Liberia,Sinoe,Tarjuwon District,Gbason Town Clinic,DAmhVvbzEl6,Gbason Town Clinic,LR5GZ3,NaN,202208,August 2022,202208,NaN,OEkmu8lM1lr,OPD new cases total,NaN,NaN,93.0,2022-11-15


In [9]:
master_table.drop(['periodid','periodname','periodcode','perioddescription','dataid','dataname',
                                  'datacode','datadescription','Total','date_downloaded'], axis=1, inplace=True)

In [10]:
master_table.drop_duplicates(subset=['orgunitlevel2','orgunitlevel3','orgunitlevel4','organisationunitid',
                                      'organisationunitname'], keep='first', inplace=True)
master_table.reset_index(inplace=True)

In [11]:
len(master_table)

848

In [12]:
print(f"Unique Level 2: {len(master_table.orgunitlevel2.unique())}")
print(f"Unique Level 3: {len(master_table.orgunitlevel3.unique())}")
print(f"Unique Level 4: {len(master_table.orgunitlevel4.unique())}")
print(f"Unique Level 5: {len(master_table.organisationunitname.unique())}")

Unique Level 2: 15
Unique Level 3: 93
Unique Level 4: 846
Unique Level 5: 846


In [13]:
print(f"Unique Geoboundaries Adm 1: {len(geob1)}")
print(f"Unique Geoboundaries Adm 2: {len(geob2)}")

Unique Geoboundaries Adm 1: 15
Unique Geoboundaries Adm 2: 136


## Match health list admin names to shapefile admin names
### Match orgunitlevel4 to Geob Adm 3
Try using fuzzy matching  

In [14]:
level4_list = list(master_table.orgunitlevel4.unique())
level4_list.sort()
level3_list = list(master_table.orgunitlevel3.unique())
level3_list.sort()
geob2_list = list(geob2.shapeName)
geob2_list.sort()

Print names to inspect

In [15]:
for letter in ascii_uppercase:
    level3_sublist = [name for name in level3_list if name[0]==letter]
    geob2_sublist = [name for name in geob2_list if name[0]==letter]

    if len(level3_sublist)>0:
        print(f"Master list level 4 ({len(level3_sublist)})")
        print("\t"+str(level3_sublist))
    if len(geob2_sublist) >0:
        print(f"Geoboundaries adm3({len(geob2_sublist)})")
        print("\t"+str(geob2_sublist))
        
    print("\n")



Master list level 4 (11)
	["B'Hai District", 'Barclayville District', 'Barrobo Farjah District', 'Barrobo Whojah District', 'Belleh District', 'Bokomu District', 'Bopolu District', 'Buah District', 'Buchanan District', 'Bushrod District', 'Butaw District']
Geoboundaries adm3(15)
	["B'hai", 'Barclayville', 'Beawor', 'Belleh', 'Bleebo', 'Bodae', 'Boe & Quilla', 'Boinsen', 'Bokomu', 'Bokon', 'Bolloh', 'Bopolu', 'Buah', 'Butaw', 'Buu-Yao']


Master list level 4 (8)
	['Campwood District', 'Careysburg District', 'Cavalla District', 'Central C District', 'Central Monrovia district', 'Chedepo District', 'Commonwealth District', 'Commonwealth-C District']
Geoboundaries adm3(7)
	['Careysburg', 'Cavala', 'Central Rivercess', 'Chedepo', 'Commonwealth', 'Commonwealth 1', 'Commonwealth 2']


Master list level 4 (9)
	['District # 1', 'District # 2', 'District # 3 A&B', 'District # 3 C', 'District # 4', 'Doedian District', 'Dorbor District', 'Dowein', 'Dugbe River District']
Geoboundaries adm3(10)
	

### Match orgunitlevel3 to Geob Adm 2

In [16]:
master_table.loc[:, "orgunitlevel3_edit"] = master_table.orgunitlevel3

In [17]:
remove_words(geob2, "shapeName", [" City Council", " District Council", " Municipal Council", " District"])
remove_words(master_table, "orgunitlevel3_edit", [" District"])

In [18]:
master_table.loc[master_table.orgunitlevel3_edit == "Chattogram", "orgunitlevel3_edit"] = "Chittagong"

In [19]:
level3_list = list(master_table.orgunitlevel3_edit.unique())
level3_list.sort()
geob2_list = list(geob2.shapeName)
geob2_list.sort()

In [20]:
for letter in ascii_uppercase:
    level3_sublist = [name for name in level3_list if name[0]==letter]
    geob2_sublist = [name for name in geob2_list if name[0]==letter]
    if len(level3_sublist)>0:
        print("Master list level 3")
        print("\t"+str(level3_sublist))
    if len(geob2_sublist) >0:
        print("Geoboundaries adm2")
        print("\t"+str(geob2_sublist))

Master list level 3
	["B'Hai", 'Barclayville', 'Barrobo Farjah', 'Barrobo Whojah', 'Belleh', 'Bokomu', 'Bopolu', 'Buah', 'Buchanan', 'Bushrod', 'Butaw']
Geoboundaries adm2
	["B'hai", 'Barclayville', 'Beawor', 'Belleh', 'Bleebo', 'Bodae', 'Boe & Quilla', 'Boinsen', 'Bokomu', 'Bokon', 'Bolloh', 'Bopolu', 'Buah', 'Butaw', 'Buu-Yao']
Master list level 3
	['Campwood', 'Careysburg', 'Cavalla', 'Central C', 'Central Monrovia district', 'Chedepo', 'Commonwealth', 'Commonwealth-C']
Geoboundaries adm2
	['Careysburg', 'Cavala', 'Central Rivercess', 'Chedepo', 'Commonwealth', 'Commonwealth 1', 'Commonwealth 2']
Master list level 3
	['District # 1', 'District # 2', 'District # 3 A&B', 'District # 3 C', 'District # 4', 'Doedian', 'Dorbor', 'Dowein', 'Dugbe River']
Geoboundaries adm2
	['District #1', 'District #2', 'District #3', 'District #4', 'Doe', 'Doedain', 'Dorbor', 'Dowein', 'Dugbe River', 'Dweh']
Master list level 3
	['Firestone', 'Foya', 'Fuamah']
Geoboundaries adm2
	['Felo-Jekwi', 'Fen Rive

In [21]:
# List for dicts for easy dataframe creation
dict_list = []
# iterating over our players without salaries found above
for name in level3_list:
    
    # Use our method to find best match, we can set a threshold here

    match = match_name(name, geob2_list, 30)
    print(f"{name}: {match}")
    # New dict for storing data
    dict_ = {}
    dict_.update({"name_level3" : name})
    dict_.update({"name_geob2" : match[0]})
    dict_.update({"score" : match[1]})
    dict_list.append(dict_)

table_adm2 = pd.DataFrame(dict_list)
table_adm2_matches = table_adm2[table_adm2.score >= 80]

Exact Match! [bhai] == [bhai]
B'Hai: ("B'hai", 100)
Barclayville vs B'hai: phon: 25, fuzzy: 35
Exact Match! [barclayville] == [barclayville]
Barclayville: ('Barclayville', 100)
Barrobo Farjah vs B'hai: phon: 22, fuzzy: 21
Barrobo Farjah vs Barclayville: phon: 46, fuzzy: 31
Barrobo Farjah vs Beawor: phon: 40, fuzzy: 40
Barrobo Farjah vs Belleh: phon: 22, fuzzy: 20
Barrobo Farjah vs Bleebo: phon: 40, fuzzy: 30
Barrobo Farjah vs Bodae: phon: 22, fuzzy: 32
Barrobo Farjah vs Boe & Quilla: phon: 33, fuzzy: 31
Barrobo Farjah vs Boinsen: phon: 18, fuzzy: 19
Barrobo Farjah vs Bokomu: phon: 20, fuzzy: 30
Barrobo Farjah vs Bokon: phon: 20, fuzzy: 32
Barrobo Farjah vs Bolloh: phon: 22, fuzzy: 40
Barrobo Farjah vs Bopolu: phon: 20, fuzzy: 30
Barrobo Farjah vs Buah: phon: 25, fuzzy: 33
Barrobo Farjah vs Butaw: phon: 22, fuzzy: 21
Barrobo Farjah vs Buu-Yao: phon: 22, fuzzy: 29
Barrobo Farjah vs Careysburg: phon: 46, fuzzy: 33
Barrobo Farjah vs Cavala: phon: 20, fuzzy: 30
Barrobo Farjah vs Central Riv

In [22]:
matches_pct = (len(table_adm2_matches) / len(level3_list))
'{:.2f}%'.format(matches_pct*100)
# old val was 77.42%

'83.87%'

### Match orgunitlevel2 to Geob Adm 1

In [23]:
master_table.loc[:, "orgunitlevel2_edit"] = master_table.orgunitlevel2

In [24]:
remove_words(master_table, "orgunitlevel2_edit", [" District"])
remove_words(geob1, "shapeName", [" City Council", " District Council", " Municipal Council", "Region"])

In [25]:
master_table.loc[master_table.orgunitlevel2_edit == "Chattogram", "orgunitlevel2_edit"] = "Chittagong"

In [26]:
level2_list = list(master_table.orgunitlevel2_edit.unique())
level2_list.sort()
geob1_list = list(geob1.shapeName)
geob1_list.sort()

In [27]:
for letter in ascii_uppercase:
    level2_sublist = [name for name in level2_list if name[0]==letter]
    geob1_sublist = [name for name in geob1_list if name[0]==letter]
    if len(level2_sublist)>0:
        print("Master list level 2")
        print("\t"+str(level2_sublist))
    if len(geob1_sublist) > 0:
        print("Geoboundaries adm1")
        print("\t"+str(geob1_sublist))
    print("\n")



Master list level 2
	['Bomi', 'Bong']
Geoboundaries adm1
	['Bomi', 'Bong']










Master list level 2
	['Gbarpolu', 'Grand Bassa', 'Grand Cape Mount', 'Grand Gedeh', 'Grand Kru']
Geoboundaries adm1
	['Gbarpolu', 'Grand Bassa', 'Grand Cape Mount', 'Grand Gedeh', 'Grand Kru']










Master list level 2
	['Lofa']
Geoboundaries adm1
	['Lofa']


Master list level 2
	['Margibi', 'Maryland', 'Montserrado']
Geoboundaries adm1
	['Margibi', 'Maryland', 'Montserrado']


Master list level 2
	['Nimba']
Geoboundaries adm1
	['Nimba']








Master list level 2
	['River Gee', 'Rivercess']
Geoboundaries adm1
	['River Gee', 'Rivercess']


Master list level 2
	['Sinoe']
Geoboundaries adm1
	['Sinoe']


















In [28]:
# List for dicts for easy dataframe creation
dict_list = []

for name in level2_list:
    
    # Use our method to find best match, we can set a threshold here
    match = match_name(name, geob1_list, 30)

    # New dict for storing data
    dict_ = {}
    dict_.update({"name_level2" : name})
    dict_.update({"name_geob1" : match[0]})
    dict_.update({"score" : match[1]})
    dict_list.append(dict_)

table_adm1 = pd.DataFrame(dict_list)
table_adm1_matches = table_adm1[table_adm1.score >= 80]

Exact Match! [bomi] == [bomi]
Bong vs Bomi: phon: 40, fuzzy: 50
Exact Match! [bong] == [bong]
Gbarpolu vs Bomi: phon: 29, fuzzy: 17
Gbarpolu vs Bong: phon: 25, fuzzy: 17
Exact Match! [gbarpolu] == [gbarpolu]
Grand Bassa vs Bomi: phon: 22, fuzzy: 13
Grand Bassa vs Bong: phon: 20, fuzzy: 13
Grand Bassa vs Gbarpolu: phon: 33, fuzzy: 21
Exact Match! [grand bassa] == [grand bassa]
Grand Cape Mount vs Bomi: phon: 15, fuzzy: 10
Grand Cape Mount vs Bong: phon: 29, fuzzy: 20
Grand Cape Mount vs Gbarpolu: phon: 38, fuzzy: 42
Grand Cape Mount vs Grand Bassa: phon: 56, fuzzy: 52
Exact Match! [grand cape mount] == [grand cape mount]
Grand Gedeh vs Bomi: phon: 0, fuzzy: 0
Grand Gedeh vs Bong: phon: 20, fuzzy: 13
Grand Gedeh vs Gbarpolu: phon: 33, fuzzy: 21
Grand Gedeh vs Grand Bassa: phon: 71, fuzzy: 55
Grand Gedeh vs Grand Cape Mount: phon: 67, fuzzy: 52
Exact Match! [grand gedeh] == [grand gedeh]
Grand Kru vs Bomi: phon: 0, fuzzy: 0
Grand Kru vs Bong: phon: 40, fuzzy: 15
Grand Kru vs Gbarpolu: pho

In [29]:
matches_pct = (len(table_adm1_matches) / len(level2_list))
'{:.2f}%'.format(matches_pct*100)

'100.00%'

Loop through each row in master table, and add new attribute names (adm3 and adm2), if the names produced matches in the lookup tables

In [30]:
master_table_copy = master_table.copy()

In [31]:
for idx, row in master_table_copy.iterrows():
    
    # if row.orgunitlevel4_edit in list(table_adm3_matches.name_level4):
    #
    #     match = table_adm3_matches.loc[table_adm3_matches.name_level4==row.orgunitlevel4_edit, "name_geob3"].iloc[0]
    #     master_table_copy.loc[idx, 'adm3'] = match
        
    if row.orgunitlevel3_edit in list(table_adm2_matches.name_level3):
        
        match = table_adm2_matches.loc[table_adm2_matches.name_level3==row.orgunitlevel3_edit, "name_geob2"].iloc[0]
        master_table_copy.loc[idx, 'adm2'] = match
        
    if row.orgunitlevel2_edit in list(table_adm1_matches.name_level2):
        
        match = table_adm1_matches.loc[table_adm1_matches.name_level2==row.orgunitlevel2_edit, "name_geob1"].iloc[0]
        master_table_copy.loc[idx, 'adm1'] = match

In [32]:
master_table_copy.adm2.isna().sum()

248

In [33]:
master_table_copy.adm1.isna().sum()

0

In [34]:
master_table_copy.head(2)

,index,orgunitlevel1,orgunitlevel2,orgunitlevel3,orgunitlevel4,organisationunitid,organisationunitname,organisationunitcode,organisationunitdescription,orgunitlevel3_edit,orgunitlevel2_edit,adm1,adm2
0,0,Liberia,Sinoe,Tarjuwon District,Gbason Town Clinic,DAmhVvbzEl6,Gbason Town Clinic,LR5GZ3,NaN,Tarjuwon,Sinoe,Sinoe,NaN
1,4,Liberia,Margibi,Mambah-Kaba District,14 Military Hospital,oUPdjvJABX7,14 Military Hospital,LR0GW8,NaN,Mambah-Kaba,Margibi,Margibi,Mambah Kaba
